In [2]:
#import cvxpy as cvx
#import cvxopt
#import networkx as nx

import argparse
#from typing import Tuple
# %matplotlib inline
#import matplotlib.pyplot as plt
#import random
#from time import time
#import os, sys
#import subprocess
#import re
import numpy as np
#import pylab
import pickle
import types
from qiskit import Aer, BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import ExactEigensolver, VQE
from qiskit.aqua.components.optimizers import SPSA, COBYLA, L_BFGS_B
from qiskit.aqua.components.variational_forms import RY, RYRZ
#from qiskit.aqua.operators import Z2Symmetries
from qiskit.chemistry import FermionicOperator
#from qiskit.chemistry.core import Hamiltonian, QubitMappingType, TransformationType
from qiskit.chemistry.drivers import PySCFDriver, UnitsType



args = types.SimpleNamespace()
args.molecule='H2' 
args.basis_set='ccpvdz' 
#args.basis_set='sto3g' 
#args.algorithm='VQE' 
#args.transformation_type='TransformationType.FULL' 
#args.qubitmapping_type='QubitMappingType.BRAVYI_KITAEV' 
args.qubitmapping_type='bravyi_kitaev'
#args.qubitmapping_type='jordan_wigner'
#args.qubitmapping_type='parity'
#args.two_qubit_reduce = True
#args.vqe_optimizer='SPSA' 
#args.vqe_max_iter=2
#args.vqe_var_form='RY' 
args.vqe_depth=1 
args.vqe_entangler='linear' 
args.num_shots=10
args.max_parallel_threads=10 
#args.vqe_sim = True
#args.vqe_aer = True
#args.datapath='/pylon5/cc5phsp/cbernaci/mol2qpu'
args.outpath_mol='/home/ubuntu/mol2qpu/output/operators/molecule/'
args.outpath_ferm='/home/ubuntu/mol2qpu/output/operators/fermionic/'
args.outpath_qub='/home/ubuntu/mol2qpu/output/operators/qubit/'
args.outpath_vqe='/home/ubuntu/mol2qpu/output/VQE_runs/'
#args.random_seed=750 
#args.vqe_opt_params = False





In [3]:
## load qmolecule
d_string = '0pt55'
filename_mol = args.outpath_mol + args.molecule + '_' + args.basis_set + '_' + d_string + '_MOLE' + '.pkl'
qmolecule = pickle.load(open(filename_mol,'rb'))

## load operators
filename_ferm = args.outpath_ferm + args.molecule + '_' + args.basis_set + '_' + d_string + '_FERM' + '.pkl'
filename_qub = args.outpath_qub + args.molecule + '_' + args.basis_set + '_' + d_string + '_QUBIT_bk' + '.pkl'
ferOp = pickle.load(open(filename_ferm, 'rb'))
qubitOp = pickle.load(open(filename_qub, 'rb'))



In [4]:
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(circuit_caching=True, 
                                   backend=backend,
                                   backend_options={'max_parallel_threads': args.max_parallel_threads,                                                                             'max_parallel_experiments': 0, 
                                                    'shots': args.num_shots})

/home/ubuntu/anaconda3/envs/mol2qpu/lib/python3.8/site-packages/qiskit/aqua/quantum_instance.py:205: UserWarning: circuit_caching will be removed at Qiskit Aqua 0.7+, this setting will be ignored. On the other hand, Qiskit Aqua does support parameterized circuits for adaptive algorithms (e.g. VQE and VQC) to avoid for compiling the circuit with the same topology multiple times
  warnings.warn("circuit_caching will be removed at Qiskit Aqua 0.7+, "


In [5]:
optimizer = SPSA(max_trials=200)
var_form = RY(qubitOp.num_qubits, depth=args.vqe_depth, entanglement=args.vqe_entangler)   

In [6]:
algo = VQE(qubitOp, var_form, optimizer)

In [ ]:
result = algo.run(quantum_instance)  

In [ ]:
filename_vqe = args.outpath_vqe + args.molecule + '_' + args.basis_set + '_' + d_string + '_VQE_bk' + '.pkl'
filehandler_vqe = open(filename_vqe, 'wb')
pickle.dump(result, filehandler_vqe)

In [15]:
num_particles = qmolecule.num_alpha + qmolecule.num_beta

print("num orbitals = ", qmolecule.num_orbitals)
print("num_particles = ", num_particles)
print("num qubits = ", qubitOp.num_qubits)
#print(dir(ferOp))
#print(dir(qubitOp))

num orbitals =  10
num_particles =  2
num qubits =  20
